In [1]:
import os
import re
import random
import pandas as pd

In [2]:
toks = [2000, 100, 200]

its = ["Transcript:['Ellie: ", "Synopsis:['The Participant", "Sentiment:['The Participant"]
items = ['transcript', 'synopsis', 'sentiment']
forms = ["Transcript:['Ellie: how is life Participant: I dont know ok Ellie: i see, anything else ']",
         "Synopsis:['Synthetic synopsis here']",
         "Sentiment:['Detailed sentiment analysis here']"]

rules = ["""Transcript: The synthetic transcript should follow the adjusted depression score of {PHQ8_Score} where Participant now has {dep}.
       Given the input information, focus on making a proactive conversation between Ellie and Participant with coherent responses and a consistent storyline in the same style of the transcript.
       The conversation must have a good balance between Ellie's and Participant's dialogue.""",
        "Synopsis: The synthetic synopsis should succinctly capture the key concerns and topics discussed in the input transcript, providing insightful and reflective observations.",
        "Sentiment: Provide a detailed sentiment analysis of the synthetic synopsis, identifying and elaborating on the specific emotions expressed."
        ]
depression = [
    "good mental health and no or minimal depression",
    "mild but bearable depression",
    "moderate depression, anxiety, or slight stress",
    "moderately severe depression, anxiety, stress and concerning negative thoughts",
    "severe depression, anxiety, stress and alarming negative thoughts"
    ]
adds = ["with a different story, background, and location from Participant's life events",
       "from the synthetic input transcript",
       "from the synthetic input transcript"]

og_prompt = """>
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    
    You are an intelligent data generation assistant tasked with creating a synthetic {item}.

    Instructions:
    1. Use the input transcript above only as a reference, but create a new {item} in fluent English {add} that matches a depression/PHQ8 score of {PHQ8_Score}.
    2. Follow this rule for the generated data:
       {rule} 
       Do not repeat any previous text. Do not cut to other prompts.
    
    Output the synthetic row in a compact JSON format on a single line without any whitespace, separators, special tokens, or special symbols. For example:
    {form}
    
    Don't copy the example or repeat yourself. Avoid any white space, line breaks, or incomprehensible text.<|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    PHQ8 Score: {PHQ8_Score}
    Depression level: Participant has {dep}.
    Input Transcript: ['{Transcript}']
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    Output {start}"""

In [3]:
data = []
train = pd.read_csv("/data/DAIC_cleaned/train.csv")
sims = 3
pref = "\'PHQ8_Score\'"
for i, d in train.iterrows():
    # Clean CSV
    pid, s = d["subject_id"], d["PHQ8_Score"]
    with open(extra_outdir + "/train.json", "w") as f: js = json.load(f)
    transcript = ' '.join([mess["content"] for mess in js["messages"]])
    sgroup = s // 5
    scores = random.sample(list(set(range(24)) - set(range(sgroup * 5, (sgroup+1) * 5))), sims)
    for score in scores:
        error = False
        row, q = [pid, score], ""
        syntrans = ""
        dep = depression[score//5]
        print(row)
        for it in range(len(its)):
            curtrans = transcript if syntrans == "" else syntrans
            tok = toks[it]
            item, rule, form = items[it], rules[it], forms[it]
            if it == 0: rule = rule.format(PHQ8_Score=score,dep=dep)
            start = its[it]
            pro = og_prompt.format(Transcript=curtrans,PHQ8_Score=score,add=adds[it],
                                   item=item,rule=rule,form=form,start=start,dep=dep)
            output = ""
            outtxt = "./DAIC_25tuned_instruct_finetune/output.txt"
            !tune run generate --config daic_eval_chain_finetune.yml prompt="{pro}" \
                max_new_tokens="{tok}" > {outtxt} 2>&1
            try:
                with open(outtxt, 'r') as f: output = f.read().rstrip().split(start)
                if len(output) == 1: raise ValueError("Error in generating output!")
                output = output[-1].replace('"', "'").replace('`', "'")
                for sy in ["<|e", ">", "]", "}"]: output = output.split(sy)[0]
                if ''.join(re.split(r'Ellie:|Participant:',output)[-4:]).strip() == "":
                    raise ValueError("Output drew a blank!")
                output = output.split("\n")[0]
                if output[-1] != "'": output += "'"
                fout = start.split("[")[-1] + output
                row.append(fout)
                if it == 0: syntrans = fout
            except:
                error = True
                break
        if not error:
            print(i, row)
            data.append(row)
        else: print("Error occurred! Starting next one...")

[303, 5]
2024-06-06:10:51:31,223 INFO     [_utils.py:33] Running InferenceRecipe with resolved config:

checkpointer:
  _component_: torchtune.utils.FullModelMetaCheckpointer
  checkpoint_dir: ./DAIC_25epoch_instruct
  checkpoint_files:
  - meta_model_10.pt
  model_type: LLAMA3
  output_dir: ./DAIC_25tuned_instruct
device: cuda
dry_multiplier: 0.8
dtype: bf16
max_new_tokens: 2000
metric_logger:
  _component_: torchtune.utils.metric_logging.DiskLogger
  log_dir: ./DAIC_25tuned_instruct
model:
  _component_: torchtune.models.llama3.llama3_8b
output_dir: ./DAIC_25tuned_instruct
prompt: "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are an intelligent\
  \ data generation assistant tasked with creating a synthetic transcript.\nInstructions:\
  \ 1. Use the input transcript above only as a reference, but create a new transcript\
  \ in completely fluent English with a completely different but coherent storyline\
  \ and background from the Participant's life events that m

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



2024-06-06:12:26:42,837 INFO     [generate.py:131] <|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an intelligent data generation assistant tasked with creating a synthetic transcript.
Instructions: 1. Use the input transcript above only as a reference, but create a new transcript in completely fluent English with a completely different but coherent storyline and background from the Participant's life events that matches a PHQ8 score of 5. Adjust the intensity of the transcript such that Participant has the following: 
   - mild but bearable depression
2. Maintain the same number of columns and ensure the data is consistent with the original schema. 3. Follow this rule for the generated data:
   - Transcript: The synthetic transcript should follow the adjusted depression score of 5 where Participant now has mild but bearable depression. Given the input information, focus on making a proactive conversation between Ellie and Participant with cohesive responses and a c


KeyboardInterrupt



In [4]:
df = pd.DataFrame(data, columns=["Participant_ID", "PHQ8_Score", "Transcript", "Synopsis", "Sentiment"])
df

,Participant_ID,PHQ8_Score,Transcript,Synopsis,Sentiment
0,303,5,"'Ellie: how are you Participant: i, i'm good...",'The Participant is a resilient and socially o...,'The Participant has mixed yet duty-oriented e...
1,303,15,'Ellie: how are you Participant: okay that's ...,'The Participant had a hard time finding physi...,'The Participant appears depressed and letharg...
2,303,19,'Ellie: how are you Participant: i'm down big...,"'The Participant describes having down days, b...",'The Participant shows a depressed and pessimi...
3,304,21,'Ellie: how are you Participant: as much as i...,"'The Participant, possessed of a calm and coll...",'The Participant is extremely downcast and dep...
4,304,2,'Ellie: how are you Participant: i'm doing go...,'The Participant is truly doing well as seen i...,'The Participant's tone is casual and relaxed....
5,304,22,'Ellie: how are you Participant: like i said ...,'The Participant is attempting to conceal thei...,'The Participant describes him or herself like...
6,310,7,'Ellie: how are you Participant: Participant:...,'The Participant presents as someone with grea...,"'The Participant describes their life as if, e..."
7,310,21,"'Ellie: how are you Participant: Participant,...",'The Participant's answer depicts very alarmin...,'The Participant's response was beclouded by t...
8,310,20,'Ellie: how are you Participant: pretty much ...,'The Participant is struggling with depression...,'The Participant's expression is one of sheer ...
9,312,12,'Ellie: how are you Participant: go away <lau...,'The Participant becomes defensive and dismiss...,"'The Participant has expressed dire, desponden..."


In [5]:
dst = "DAIC_transcripts"
os.makedirs(dst, exist_ok=True)
df.to_csv(dst + "/DAIC_transcript_instruct_3ke11_test.csv", index=False)
# old_df = pd.read_csv(dst + "/DAIC_transcript_instruct_3ke3_early.csv", index_col=0)
# df = pd.concat([old_df, df], ignore_index=True)
# df.to_csv(dst + "/DAIC_transcript_instruct_3ke3.csv", index=False)